In [14]:
import pandas as pd
import json

In [15]:
# Load the CSV file
csv_file_path = "assets/data/Lab Member Info Survey Responses.csv"  # Update with correct path
df_csv = pd.read_csv(csv_file_path)

# Display the first few rows to understand its structure
df_csv.head()

,時間戳記,姓名（中英文皆可，中文為優先，例如：王小明）,系級（例如：112）,主要聯絡方式（請放常用Email，例如：nchu@gmail.com）,個人照片檔案名稱（例如：112王小明.png），另外請將照片寄給該屆網管,次要聯絡方式（請放FB或Linkedin連結）（若無，請留空白）,YouTube連結（若無，請留空白）,個人Github連結（若無，請留空白）,其他連結（例如：個人網站等）（若無，請留空白）,一段關於自己的描述（約30字左右，可以放研究領域等）,"關於你的關鍵字（約兩到三個，請用逗號隔開，例如：wsn, algorithm）",年級或職業稱謂\n1. 在校生：年級（例如：碩二）\n2. 已畢業：頭銜（例如：聯發科副理）,文憑
0,2025/3/27 下午 2:00:05,陳昱丞,112,scdev0210@gmail.com,陳昱丞.png,https://www.linkedin.com/in/yu-cheng-chen-460a...,https://www.youtube.com/@scdev-0210,https://github.com/sc0210,https://sc0210.netlify.app/,Focused on developing Metaheuristic algorithm ...,"wsn,algorithm",碩二,碩士
1,2025/3/27 下午 2:25:56,劉冠伯,113,g113056116@smail.nchu.edu.tw,113劉冠伯.png,https://www.linkedin.com/in/aidenliu,NaN,https://github.com/waterbear-tw,NaN,Engaged in reinforcement learning and resource...,NaN,碩一,碩士
2,2025/3/27 下午 2:52:16,羅泓喆,112,g112056002@mail.nchu.edu.tw,能沿用嗎?,NaN,NaN,NaN,NaN,NaN,"lidar,點雲分割",碩二,碩士
3,2025/3/27 下午 3:51:30,陳信宇,113,andy0413@hotmail.com.tw,113陳信宇.png,NaN,NaN,NaN,NaN,NaN,wsn,碩一,碩士
4,2025/3/27 下午 4:02:10,陳珮禎,112,penny5462254@gmail.com,已有,NaN,NaN,NaN,NaN,Combining federated learning with Bayesian opt...,"NILM,FL",在校生碩二,碩士


In [16]:
# Rename columns if necessary (Ensure they match the JSON keys)
df = df_csv.rename(columns={
    "姓名（中英文皆可，中文為優先，例如：王小明）": "name",
    "系級（例如：112）": "grade",
    "主要聯絡方式（請放常用Email，例如：nchu@gmail.com）": "email",
    "個人照片檔案名稱（例如：112王小明.png），另外請將照片寄給該屆網管": "img",
    "次要聯絡方式（請放FB或Linkedin連結）（若無，請留空白）": "fblinks",
    "YouTube連結（若無，請留空白）": "ytlinks",
    "個人Github連結（若無，請留空白）": "ghlinks",
    "其他連結（例如：個人網站等）（若無，請留空白）": "blinks",
    "一段關於自己的描述（約30字左右，可以放研究領域等）": "description",
    "關於你的關鍵字（約兩到三個，請用逗號隔開，例如：wsn, algorithm）": "keyword",
    "年級或職業稱謂\n1. 在校生：年級（例如：碩二）\n2. 已畢業：頭銜（例如：聯發科副理）": "title",
    "文憑": "degree"
})

# Fill NaN values with empty string for string fields
string_fields = ["name", "email", "img", "fblinks", "ytlinks", "ghlinks", "blinks", "description", "title", "degree"]

df[string_fields] = df[string_fields].fillna("")

# Process degree
degree_map = {"碩士": "master", "博士": "phd", "學士": "bachelor"}
df["degree"] = df["degree"].map(degree_map).fillna(df["degree"])

# Process 'keyword' field: split by comma and strip spaces, keep empty list if no data
df["keyword"] = df["keyword"].apply(lambda x: [kw.strip() for kw in str(x).split(",") if kw.strip()] if pd.notna(x) else [])

# Ensure 'grade' is an integer where applicable, default to 0 if missing
df["grade"] = pd.to_numeric(df["grade"], errors="coerce").fillna(0).astype(int)

# Add image path dynamically
df["img"] = df["grade"].astype(str) + df["name"] + ".png"
df["img"] = "assets/img/member/" + df["img"]


In [17]:
# Convert DataFrame to a list of dictionaries
json_data = df.to_dict(orient="records")

# Remove empty fields dynamically from each entry
cleaned_json_data = [{k: v for k, v in entry.items() if v not in ["NaN", None]} for entry in json_data]

# Sort the JSON output by 'grade' in descending order
sorted_json_data = sorted(cleaned_json_data, key=lambda x: x["grade"], reverse=True)

# Export to JSON file
output_file = "assets/data/lab_members.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(sorted_json_data, file, ensure_ascii=False, indent=2)

print(f"JSON file exported successfully as {output_file}")

JSON file exported successfully as assets/data/lab_members.json
